In [ ]:
# ALEXNET< SQUEEZENET, densenet , espnet,  
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import MobileNetV2 #(MobileNet, MobileNetV2, VGG16, VGG19)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# Load the dataset
dataset = tf.keras.preprocessing.image_dataset_from_directory( '/kaggle/input/mini-plant-village',shuffle=True,
    image_size=(256, 256),
    batch_size=32
)

# Get the class names
class_names = dataset.class_names

In [ ]:
# Display sample images from the dataset
plt.figure(figsize=(10, 10))
for image_batch, label_batch in dataset.take(1):
    for i in range(12):
        ax = plt.subplot(3, 4, i + 1)
        plt.imshow(image_batch[i].numpy().astype("uint8"))
        plt.title(class_names[label_batch[i]])
        plt.savefig("Images_MobileNet.png")

# Define the model
input_shape = (256, 256, 3)
num_classes = len(class_names)

In [ ]:
channels = 3
epochs = 50


def get_dataset_partitions_tf(dataset, train_split=0.75, val_split=0.15, test_split=0.10, shuffle=10000):
    # Compute the size of each partition
    dataset_size = len(dataset)
    train_size = int(train_split * dataset_size)
    val_size = int(val_split * dataset_size)
    test_size = int(test_split * dataset_size)

    # Create partitions
    train_ds = dataset.take(train_size)
    remaining_ds = dataset.skip(train_size)
    val_ds = remaining_ds.take(val_size)
    test_ds = remaining_ds.skip(val_size)

    return train_ds, val_ds, test_ds


train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

# GPU ) train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
train_ds = train_ds.cache().shuffle(1000)
val_ds = val_ds.cache().shuffle(1000)
test_ds = test_ds.cache().shuffle(1000)

# resize_rescale = tf.keras.Sequential([
#     layers.experimental.preprocessing.Rescaling(1.0/255)
# ])

# data_augmentation  = tf.keras.Sequential([
#     layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
#     layers.experimental.preprocessing.RandomRotation(0.2),
# ])


In [ ]:
# Load the MobileNet base model without pretrained weights
base_model = MobileNetV2(
    input_shape=input_shape,
    include_top=False,
    weights="imagenet"
)

# Freeze the base model
base_model.trainable = False

# Create the model
model = Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
history = model.fit(train_ds,
                    epochs=epochs,
                    validation_data=val_ds)

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

In [ ]:
plt.figure(figsize=(8,8))
# plt.subplot(1,2,1)
plt.plot(range(epochs),acc, label='Training Accuracy')
plt.plot(range(epochs),val_acc, label='Validation Accuracy')
plt.ylim([0, 1.1])
yticks = [i/10 for i in range(11)]
plt.legend(loc='lower right')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Training Accuracy vs Validation Accuracy")
plt.savefig("Accuracy_MobileNetV2.png")

In [ ]:
plt.figure(figsize=(8,8))
# plt.subplot(1,2,1)
plt.plot(range(epochs),loss, label='Training Loss')
plt.plot(range(epochs),val_loss, label='Validation Loss')
plt.ylim([-0.5, 10])
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Training Loss vs Validation Loss")
plt.savefig("loss_MobileNetV2.png")

In [ ]:
def predict(model,img):
    img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tf.expand_dims(img_array,0)
    
    prediction = model.predict(img_array)
    predicted_class =  class_names[np.argmax(prediction[0])]
    confidence = round(100*(np.max(prediction[0])),2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15,15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3,3,i+1)
        plt.imshow(images[i].numpy().astype("uint8"))
        predicted_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]
        plt.title(f"Actual:{actual_class},\n Predicted: {predicted_class}.")
        plt.axis("off")
        plt.savefig("Results_MobileNetV2")

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(1,2,1)
plt.plot(range(epochs),acc, label='Training Accuracy')
plt.plot(range(epochs),val_acc, label='Validation Accuracy')
plt.ylim([0, 1.1])
plt.legend(loc='lower right')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Training Accuracy vs Validation Accuracy")

plt.subplot(1,2,2)
plt.plot(range(epochs),loss, label='Training Loss')
plt.plot(range(epochs),val_loss, label='Validation Loss')
plt.ylim([-0.05, 10])
plt.legend(loc='upper right')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title("Training Loss vs Validation Loss")
plt.savefig("combined_MobileNetV2.png")


In [ ]:
    _, test_accuracy = model.evaluate(test_ds)
    print(f"Test Accuracy  :{i+1} {test_accuracy}")

In [ ]:
plt.figure(figsize=(8,8))
# plt.subplot(1,2,1)
plt.plot(range(epochs),acc, label='Training Accuracy')
plt.plot(range(epochs),val_acc, label='Validation Accuracy')
plt.ylim([0.45, 1.0])
yticks = [i/10 for i in range(11)]
plt.legend(loc='lower right')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title("Training Accuracy vs Validation Accuracy")
plt.savefig("Accuracy_1_MobileNetV2.png")

In [ ]:
# Plot the training and validation loss
plt.figure(figsize=(10, 5))
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim(0, 3)  # Limiting y-axis from 0 to 5
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.savefig("loss_1_MobileNetV2.png")  # Save the figure as loss_1.png
plt.show()


In [ ]:
from sklearn.metrics import classification_report

# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_ds)

# Predict the classes for the test set
y_pred = np.argmax(model.predict(test_ds), axis=1)

# Get the true classes for the test set
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Print the classification report
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
from sklearn.metrics import classification_report

# Make predictions on the test dataset
y_pred = []
y_true = []

for images, labels in test_ds:
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    y_pred.extend(predicted_classes)
    y_true.extend(labels.numpy())

# Generate classification report
print(classification_report(y_true, y_pred, target_names=class_names))


In [ ]:
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
from PIL import Image

# Make predictions on the test dataset
y_pred = []
y_true = []

for images, labels in test_ds:
    predictions = model.predict(images)
    predicted_classes = np.argmax(predictions, axis=1)
    y_pred.extend(predicted_classes)
    y_true.extend(labels.numpy())

# Generate classification report
report = classification_report(y_true, y_pred, target_names=class_names, output_dict=True)

# Extract data from the report
report_data = []
report_data.append(["Class", "Precision", "Recall", "F1-score"])
for class_name in class_names:
    precision = report[class_name]["precision"]
    recall = report[class_name]["recall"]
    f1_score = report[class_name]["f1-score"]
    report_data.append([class_name, precision, recall, f1_score])

# Plot the classification report
fig, ax = plt.subplots(figsize=(8, 6))
ax.axis('off')
ax.axis('tight')
ax.table(cellText=report_data,
         cellLoc='left',
         loc='center',
         edges='closed',
         bbox=[0, 0, 1, 1])
plt.savefig("classification_report_MobileNetV2.png", bbox_inches='tight', pad_inches=0, facecolor='white')
plt.close()

# Open the saved image and convert it to black text on white background
img = Image.open("classification_report_MobileNetV2.png")
img = img.convert("RGB")
img_array = np.array(img)
img_array[img_array == 255] = 0  # Convert white pixels to black
img_array[img_array < 255] = 255  # Convert remaining pixels to white
img = Image.fromarray(img_array)

# Save the final image
img.save("classification_report_final_MobileNetV2.png")


In [ ]:
import tensorflow as tf

# Directory where you want to save the TFRecord files
output_dir = "/kaggle/working/test_dataset_tfrecordMobileNetV2"

# Define function to serialize data and write to TFRecord file
def serialize_example(image, label):
    feature = {
        'image': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.serialize_tensor(image).numpy()])),
        'label': tf.train.Feature(int64_list=tf.train.Int64List(value=[label]))
    }
    example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
    return example_proto.SerializeToString()

# Create TFRecord writer
tf_record_writer = tf.io.TFRecordWriter(output_dir)

# Iterate through the dataset and serialize examples
for images, labels in test_ds:
    for image, label in zip(images, labels):
        example = serialize_example(image, label)
        tf_record_writer.write(example)

# Close the TFRecord writer
tf_record_writer.close()

print("TFRecord files saved successfully_MobileNetV2.")


In [ ]:
# Evaluate the model on the test set
loss, top1_accuracy = model.evaluate(test_ds)
print("Top-1 Accuracy:", top1_accuracy)

# Extract true labels from the test dataset
y_true = np.concatenate([y for x, y in test_ds], axis=0)

# Predict probabilities
y_prob = model.predict(test_ds)

# Calculate top-5 accuracy
def top_k_accuracy(y_true, y_prob, k=5):
    top_k = tf.nn.in_top_k(y_true, y_prob, k)
    return np.mean(top_k)

top5_accuracy = top_k_accuracy(y_true, y_prob)
print("Top-5 Accuracy:", top5_accuracy)


In [ ]:
# model_version=1
a = model.save('MobileNetV2.h5')
# Save the model
model.save("my_model")